# Sampling Data in Python


## Basic Sampling

In [ ]:
# Take 10 rows as sample
pts_vs_flavor_samp = pts_vs_flavor_pop.sample(n=10)


### Sampling Bias
Convenience sampling—collecting data using the easiest method—can result in samples that aren't representative of the population.
Visualizing the distributions of the population and the sample can help determine whether or not the sample is representative of the population.


In [ ]:
# Visualize the hist of the whole dataset and a sample of the dataset
coffee_ratings_first10["total_cup_points"].hist(bins=np.arange(59,93, 2)) #! arange will stop at 91

### Pseudo-random number generation


In [ ]:
# Generate random numbers from a Uniform(-3, 3)
uniforms = np.random.uniform(low=-3, high=3, size=5000)

# Generate random numbers from a Normal(5, 2)
normals = np.random.normal(loc=5,scale=2, size=5000)